# Data Ingestion

In [1]:
from langchain_community.document_loaders import TextLoader

In [2]:
loader = TextLoader("exmples.txt")
text_docs = loader.load()
print(text_docs)

[Document(metadata={'source': 'exmples.txt'}, page_content='This is a small sample document for testing RAG pipelines.\nIt contains a few sentences about LangChain and retrieval.\nYou can add more lines to improve chunking and recall.\nRAG systems fetch relevant text before generating answers.\n')]


In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [5]:
llm = ChatOpenAI(
    model = "arcee-ai/trinity-large-preview:free",
    temperature = 0.9,
    base_url = "https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8000",
        "X-Title": "RAG with Langchain"
    }
)

In [7]:
from langchain_community.document_loaders  import WebBaseLoader
import bs4

## Load CHUNK AND INDEX:

loader = WebBaseLoader(
    web_path="https://www.geeksforgeeks.org/artificial-intelligence/what-is-artificial-intelligence-ai/",
    bs_kwargs={"parse_only": bs4.SoupStrainer(
        class_=["ArticleHeader_article-title__futDC", "MainArticleContent_articleMainContentCss__b_1_R article--viewer_content", "html-chunk"]
    )}
)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
docs = loader.load()
print(docs[0].page_content)

What is Artificial Intelligence (AI)Artificial Intelligence (AI) is a technology that enables machines and computers to perform tasks that typically require human intelligence. It helps systems learn from data, recognize patterns and make decisions to solve complex problems. It is used in healthcare, finance, e-commerce and transportation offering personalized recommendations and enabling self-driving cars.Core Concepts of AIAI is based on core concepts and technologies that enable machines to learn, reason and make decisions on their own. Let's see some of the concepts:1. Machine Learning (ML)Machine Learning is a subset of artificial intelligence (AI) that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, a machine learning model uses algorithms to identify patterns within data and improve its performance over time without human intervention.2. Generative AIGenerative AI is designed to create ne

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

path = "AI-for-Education-RAG.pdf"
loader = PyPDFLoader(path)
docs = loader.load()

In [21]:
print(len(docs))

18


# Data Splitting

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter


In [28]:
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = recursive_splitter.split_documents(docs)
print(len(documents))

18


In [30]:
for i in documents:
    print(i.page_content)

Using your own content in LLM’s -
Retrieval Augmented Generation
(RAG)
Gen-AI can help with education in many ways
Saves time
• AI lesson planning reduces the time teachers spend preparing lessons. 
• Can automate marking using AI – image recognition or voice technologies to assess students as 
they read aloud.
Improved quality
• Adapt high quality resources to your context (languages or images).
• Can help teachers clarify concepts they may have forgotten.
• Adapt lessons/examples to children’s previous answers.
Improved scalability
• Access on phones (e.g. chatbot to support teachers) enables wide adoption. 
• Rapid integration of new national policies, curricula and best practices by automatically updating 
knowledge bases of LLMs.
But many of these require introducing your own 
information into the models. 
The training data for major LLM’s isn’t specific to any one country and often 
doesn’t include your education resources
If your content isn’t in the training data it can only ev

In [31]:
print(documents[0].page_content)
print(documents[1].page_content)

Using your own content in LLM’s -
Retrieval Augmented Generation
(RAG)
Gen-AI can help with education in many ways
Saves time
• AI lesson planning reduces the time teachers spend preparing lessons. 
• Can automate marking using AI – image recognition or voice technologies to assess students as 
they read aloud.
Improved quality
• Adapt high quality resources to your context (languages or images).
• Can help teachers clarify concepts they may have forgotten.
• Adapt lessons/examples to children’s previous answers.
Improved scalability
• Access on phones (e.g. chatbot to support teachers) enables wide adoption. 
• Rapid integration of new national policies, curricula and best practices by automatically updating 
knowledge bases of LLMs.


In [32]:
character_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
character_documents = character_splitter.split_documents(docs)
print(len(character_documents))

18


In [33]:
for i in range(5):
    print(documents[i].page_content)

Using your own content in LLM’s -
Retrieval Augmented Generation
(RAG)
Gen-AI can help with education in many ways
Saves time
• AI lesson planning reduces the time teachers spend preparing lessons. 
• Can automate marking using AI – image recognition or voice technologies to assess students as 
they read aloud.
Improved quality
• Adapt high quality resources to your context (languages or images).
• Can help teachers clarify concepts they may have forgotten.
• Adapt lessons/examples to children’s previous answers.
Improved scalability
• Access on phones (e.g. chatbot to support teachers) enables wide adoption. 
• Rapid integration of new national policies, curricula and best practices by automatically updating 
knowledge bases of LLMs.
But many of these require introducing your own 
information into the models. 
The training data for major LLM’s isn’t specific to any one country and often 
doesn’t include your education resources
If your content isn’t in the training data it can only ev